In [1]:
#%cd /content/drive/My Drive/SegFormer
import argparse
import logging
import os
import random
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from utils import clip_gradient, AvgMeter

from glob import glob
from skimage.io import imread
import matplotlib.pyplot as plt
import pandas as pd
from collections import OrderedDict
from torch.autograd import Variable
from datetime import datetime
import torch.nn.functional as F
import cv2
from albumentations.augmentations import transforms
from albumentations.core.composition import Compose, OneOf
import math

/home/s/anaconda3/envs/tf2.6/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import argparse
import copy
import os
import os.path as osp
import time

import mmcv
import torch
from mmcv.runner import init_dist
from mmcv.utils import Config, DictAction, get_git_hash

from mmseg import __version__
from mmseg.apis import set_random_seed, train_segmentor
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import collect_env, get_root_logger

In [3]:
from mmseg.models.segmentors import CaraSegUPer_ver2 as UNet
from mmseg.models.segmentors import CaraSegUPer_wBiRAFPN as Net2
from mmseg.models.segmentors import CaraSegUPer_wBiFPN as Net
from mmseg.models.segmentors import CaraSegUPer_wBiFPN_IGH as IGH

In [5]:
HEIGHT=384
WIDTH = 384

In [12]:
from PIL import Image
from keras.utils.np_utils import to_categorical   
# Convert numpy data to tensorflow data
'''
We need segent 3 classes:
    + 0 if the pixel is part of the image background (denoted by black color);
    + 1 if the pixel is part of a non-neoplastic polyp (denoted by green color);
    + 2 if the pixel is part of a neoplastic polyp (denoted by red color).
'''
import random
from imgaug import augmenters as iaa
from keras.preprocessing.image import ImageDataGenerator
sometimes = lambda aug: iaa.Sometimes(.5, aug)   
aug_pipe = iaa.Sequential(
            [      


                iaa.SomeOf((0, 3),
                    [
                       # sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
                        #iaa.OneOf([
                        #    iaa.GaussianBlur((0, 1.5)), # blur images with a sigma between 0 and 3.0
                        #    iaa.AverageBlur(k=(2,5)), # blur image using local means with kernel sizes between 2 and 7
                        #    iaa.MedianBlur(k=(3, 5)), # blur image using local medians with kernel sizes between 2 and 7
                        #]),
                        iaa.Sharpen(alpha=(0, 0.02), lightness=(0.95, 1.05)), # sharpen images
                        
                       iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.02*255), per_channel=0.5), # add gaussian noise to images
                       
                        
                        iaa.Add((-10, 10), per_channel=0.5), 
                        iaa.Multiply((0.95, 1.05), per_channel=0.5), 
                        iaa.ContrastNormalization((0.95, 1.05), per_channel=0.5), # improve or worsen the contrast
                    ],
                    random_order=True
                )
            ],
            random_order=True
        )
image_datagen_args = {
		'shear_range': 0.1,
		'zoom_range': 0.3,
		'width_shift_range': 0.25,
		'height_shift_range': 0.25,
		'rotation_range': 180,
		'horizontal_flip': True,
		'vertical_flip': True,
        'fill_mode':'constant'
	}
image_datagen = ImageDataGenerator(**image_datagen_args)
def augment(image,mask):
    #image *= 255
    image = image.astype(np.uint8)
    if random.random()<0.5:
        seed = random.randint(0,1000000000)
        params = image_datagen.get_random_transform(image.shape,seed = seed)
        image = image_datagen.apply_transform(image, params)
        params = image_datagen.get_random_transform(mask.shape,seed = seed)
        mask = image_datagen.apply_transform(np.expand_dims(mask,-1), params)[:,:,0]
    if random.random()<0.5:
        image = aug_pipe.augment_image(np.array(image).astype(np.uint8)) 
    #image = image/255.
    return image.astype(np.float32),mask

# def read_image(image_path):
#     image = cv2.imread(image_path, cv.IMREAD_COLOR)
#     image = cv2.resize(image, (WIDTH, HEIGHT))
#     image = image/255.0
#     image = image.astype(np.float32)
#     return image
# def read_mask(mask_path):
#     """
#     image = cv.imread(mask_path,cv2.IMREAD_GRAYSCALE)
    
#     image = cv.resize(image, (WIDTH, HEIGHT), interpolation = cv2.INTER_NEAREST)
#     image[image>0]=1
#     """
#     with open(mask_path, 'rb') as f:
#             img = Image.open(f)
#             img = img.convert('L')
#     image = np.array(img)
#     image = cv.resize(image, (WIDTH, HEIGHT), interpolation = cv2.INTER_NEAREST)/255
    
#     return preprocess_mask(image.astype(np.uint8))

def pad_image(image,mask):
    
    h,w,c = image.shape
    h_,w_ = mask.shape
    assert (h==h_) and (w==w_)
    final = np.zeros((HEIGHT,HEIGHT,3))
    final_mask = np.zeros((HEIGHT,HEIGHT))
    if h == w: return image,mask
    if h>w:
        new_w = w*HEIGHT//h
        image = cv2.resize(image,(new_w,HEIGHT))
        mask = cv2.resize(mask,(new_w,HEIGHT),interpolation =cv2.INTER_NEAREST)
        final[:,(HEIGHT-new_w)//2:(HEIGHT-new_w)//2+new_w,:] = image
        final_mask[:,(HEIGHT-new_w)//2:(HEIGHT-new_w)//2+new_w] = mask
    else:
        new_h = h*HEIGHT//w
        image = cv2.resize(image,(HEIGHT,new_h))
        mask =  cv2.resize(mask,(HEIGHT,new_h))
        final[(HEIGHT-new_h)//2:(HEIGHT-new_h)//2+new_h,:,:] = image 
        final_mask[(HEIGHT-new_h)//2:(HEIGHT-new_h)//2+new_h,:] = mask 
    #image = cv2.resize(image, (HEIGHT, HEIGHT))
   # image = image/255.0
  #  image = image.astype(np.float32)
    return final.astype(np.uint8), final_mask.astype(np.uint8)

def random_pad(image,mask ):
    h,w,_ = image.shape
    if random.random() < 0.2:
        if random.random()<0.5:
            anchor_tlx = random.randint(0,w//4)
            anchor_brx = random.randint(3*w//4,w-1)
            image[:,:anchor_tlx,:] =0
            image[:,anchor_brx:,:] = 0
            mask[:,:anchor_tlx] =0
            mask[:,anchor_brx:] =0
        else  : 
            anchor_tly = random.randint(0,h//4)
            anchor_bry = random.randint(3*h//4,h-1)
            image[:,:anchor_tly,:] =0
            image[:,anchor_bry:,:] = 0
            mask[:,:anchor_tly] =0
            mask[:,anchor_bry:] =0
    return image,mask
class Dataset(torch.utils.data.Dataset):

    def __init__(self, img_paths, mask_paths, aug=True, transform=None):
        self.img_paths = img_paths
        self.mask_paths = mask_paths
        self.aug = aug
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        mask_path = self.mask_paths[idx]
        # image = imread(img_path)
        # mask = imread(mask_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, 0)
        # name = self.img_paths[idx].split('/')[-1]

        if self.transform is not None:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
            image = cv2.resize(image, (384, 384))
            mask = cv2.resize(mask, (384, 384)) 
        else:
            image = cv2.resize(image, (384, 384))
            mask = cv2.resize(mask, (384, 384)) 

        image = image.astype('float32') / 255
        image = image.transpose((2, 0, 1))

        mask = mask[:,:,np.newaxis]
        mask = mask.astype('float32') / 255
        mask = mask.transpose((2, 0, 1))

        return np.asarray(image), np.asarray(mask)

In [13]:
from keras import backend as K

def recall_m(y_true, y_pred):
  true_positives = torch.sum(torch.round(torch.clip(y_true * y_pred, 0, 1)))
  possible_positives = torch.sum(torch.round(torch.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = torch.sum(torch.round(torch.clip(y_true * y_pred, 0, 1)))
  predicted_positives = torch.sum(torch.round(torch.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here

def dice_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

def iou_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return recall*precision/(recall+precision-recall*precision +K.epsilon())

In [14]:
class FocalLossV1(nn.Module):

    def __init__(self,
                 alpha=0.25,
                 gamma=2,
                 reduction='mean',):
        super(FocalLossV1, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.crit = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, logits, label):
        # compute loss
        logits = logits.float() # use fp32 if logits is fp16
        with torch.no_grad():
            alpha = torch.empty_like(logits).fill_(1 - self.alpha)
            alpha[label == 1] = self.alpha

        probs = torch.sigmoid(logits)
        pt = torch.where(label == 1, probs, 1 - probs)
        ce_loss = self.crit(logits, label.float())
        loss = (alpha * torch.pow(1 - pt, self.gamma) * ce_loss)
        if self.reduction == 'mean':
            loss = loss.mean()
        if self.reduction == 'sum':
            loss = loss.sum()
        return loss
    
class TverskyLoss(nn.Module):
    def __init__(self, alpha=0.7, eps=1):
        super().__init__()

        self.eps = eps
        self.alpha = alpha

    def forward(self, pred, label):
        """
        Forward function
        :param pred: Prediction tensor containing raw network outputs (no logit) (B x C x H x W)
        :param label: Label mask tensor (B x C x H x W)
        """
        probs = torch.sigmoid(pred)
        true_pos = torch.sum(probs * label, dim=[0, 2, 3])
        false_neg = torch.sum(label * (1 - probs), dim=[0, 2, 3])
        false_pos = torch.sum(probs * (1 - label), dim=[0, 2, 3])
        return 1 - torch.mean(
            (true_pos + self.eps)
            / (
                true_pos
                + self.alpha * false_neg
                + (1 - self.alpha) * false_pos
                + self.eps
            )
        )

class FocalTverskyLoss(TverskyLoss):
    def __init__(self, gamma=4 / 3, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma

    def forward(self, pred, label):
        probs = torch.sigmoid(pred)
        true_pos = torch.sum(probs * label, dim=[0, 2, 3])
        false_neg = torch.sum(label * (1 - probs), dim=[0, 2, 3])
        false_pos = torch.sum(probs * (1 - label), dim=[0, 2, 3])

        t = (true_pos + self.eps) / (
            true_pos + self.alpha * false_neg + (1 - self.alpha) * false_pos + self.eps
        )

        x = torch.pow(1 - t, 1 / self.gamma)

        return x #torch.sum(x)
class FocalLoss(nn.CrossEntropyLoss):
    ''' Focal loss for classification tasks on imbalanced datasets '''

    def __init__(self, gamma, alpha=None, ignore_index=-100, reduction='mean'):
        super().__init__(weight=alpha, ignore_index=ignore_index, reduction='none')
        self.reduction = reduction
        self.gamma = gamma

    def forward(self, input_, target):
        cross_entropy = super().forward(input_, target)
        # Temporarily mask out ignore index to '0' for valid gather-indices input.
        # This won't contribute final loss as the cross_entropy contribution
        # for these would be zero.
        target = torch.argmax(target,dim=1)
        target = target * (target != self.ignore_index).long()
        input_prob = torch.gather(F.softmax(input_, 1), 1, target.unsqueeze(1))
        loss = torch.pow(1 - input_prob, self.gamma) * cross_entropy
        return torch.mean(loss) if self.reduction == 'mean'\
               else torch.sum(loss) if self.reduction == 'sum'\
               else loss    
def structure_loss_v2(pred, mask):
    weit = 1 + 5*torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    wfocal = FocalLossV1()(pred, mask)
    wfocal = (wfocal*weit).sum(dim=(2,3)) / weit.sum(dim=(2, 3))
    wiou = FocalTverskyLoss()(pred,mask)
    return (wfocal + wiou).mean()

def structure_loss(pred, mask):
    weit = 1 + 5*torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    wfocal = FocalLossV1()(pred, mask)
    wfocal = (wfocal*weit).sum(dim=(2,3)) / weit.sum(dim=(2, 3))

    pred = torch.sigmoid(pred)
    inter = ((pred * mask)*weit).sum(dim=(2, 3))
    union = ((pred + mask)*weit).sum(dim=(2, 3))
    wiou = 1 - (inter + 1)/(union - inter+1)
    return (wfocal + wiou).mean()
def structure_loss_(pred, mask):
    loss=FocalLoss(gamma=2)#nn.CrossEntropyLoss()#
    return loss(pred, mask)
def structure_loss__(pred, mask):
    l1 = structure_loss__(torch.unsqueeze(pred[:,1,:,:],1),torch.unsqueeze(mask[:,1,:,:],1))
    l2 = structure_loss__(torch.unsqueeze(pred[:,2,:,:],1),torch.unsqueeze(mask[:,2,:,:],1))
    return l1 + l2

In [15]:
import sys
def train(train_loader, model, optimizer, epoch, lr_scheduler, deep=False):
    model.train()
    # ---- multi-scale training ----
    size_rates =[256/HEIGHT, 1, 512/HEIGHT]
    loss_record = AvgMeter()
    dice, iou = AvgMeter(), AvgMeter()
    for i, pack in enumerate(train_loader, start=1):
        if epoch <= 1:
                optimizer.param_groups[0]["lr"] = (epoch * i) / (1.0 * total_step) * init_lr
        else:
            lr_scheduler.step()

        for rate in size_rates: 
            optimizer.zero_grad()
            # ---- data prepare ----
            images, gts = pack
            images = Variable(images).cuda()
            gts = Variable(gts).cuda()
            # ---- rescale ----
            trainsize = int(round(trainsize_init*rate/32)*32)
            images = F.upsample(images, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            gts = F.upsample(gts, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            # ---- forward ----
            map4, map3, map2, map1 = model(images)
           # print(map4.shape, map3.shape, map2.shape, map1.shape)
            map1 = F.upsample(map1, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            map2 = F.upsample(map2, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            map3 = F.upsample(map3, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            map4 = F.upsample(map4, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            #print(map1.shape)
            loss = structure_loss(map1, gts) + structure_loss(map2, gts) + structure_loss(map3, gts) + structure_loss(map4, gts)
            # ---- metrics ----i
            dice_score = dice_m(map1, gts)
            iou_score = iou_m(map1, gts)
            # ---- backward ----
            loss.backward()
            clip_gradient(optimizer, clip)
            optimizer.step()
            # ---- recording loss ----
            if rate == 1:
                 loss_record.update(loss.data, batchsize)
                 dice.update(dice_score.data, batchsize)
                 iou.update(iou_score.data, batchsize)
        sys.stdout.write("\rloss" +str(loss_record.show()) + ",   dice: "+str(dice.show())+"  ,iou:"+str(iou.show()))
        sys.stdout.flush()
       
        # ---- train visualization ----
        if i == total_step:
            print('{} Training Epoch [{:03d}/{:03d}], '
                  '[loss: {:0.4f}   dice:{:0.4f}  iou: {:0.4f}]'.
                  format(datetime.now(), epoch, num_epochs,\
                         loss_record.show(), dice.show(), iou.show()))

    ckpt_path = save_path + 'last.pth'
    print('[Saving Checkpoint:]', ckpt_path)
    checkpoint = {
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': lr_scheduler.state_dict()
    }
    torch.save(checkpoint, ckpt_path)

    log = OrderedDict([
        ('loss', loss_record.show()),('dice', dice.show()), ('iou',iou.show())
    ])

    return log

In [26]:
def recall_np(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_np(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def dice_np(y_true, y_pred):
    precision = precision_np(y_true, y_pred)
    recall = recall_np(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def iou_np(y_true, y_pred):
    intersection = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    union = np.sum(y_true)+np.sum(y_pred)-intersection
    return intersection/(union+K.epsilon())

def get_scores(gts, prs):
    mean_precision = 0
    mean_recall = 0
    mean_iou = 0
    mean_dice = 0
    for gt, pr in zip(gts, prs):
        mean_precision += precision_np(gt, pr)
        mean_recall += recall_np(gt, pr)
        mean_iou += iou_np(gt, pr)
        mean_dice += dice_np(gt, pr)

    mean_precision /= len(gts)
    mean_recall /= len(gts)
    mean_iou /= len(gts)
    mean_dice /= len(gts)        
    
    print(f"scores: dice={mean_dice}, miou={mean_iou}, precision={mean_precision}, recall={mean_recall}")

    return (mean_iou, mean_dice, mean_precision, mean_recall)



def inference(model,writer=None,epoch=None,list_=[]):
    print("#"*20)
    model.eval()
    kernel = np.ones((2,2), np.uint8)
    dataset_name = 'stage1 test'
    if True:
#     dataset_names = ['Kvasir', 'CVC-ClinicDB', 'CVC-ColonDB', 'CVC-300', 'ETIS-LaribPolypDB']
#     for dataset_name in dataset_names:
        data_path = f'../dataset/IGH-small/train/train/*'
        print(data_path)
        X_test =  glob('../dataset/CHASE/CHASE_Test_8/images/*')
        X_test.sort()
        y_test = glob('../dataset/CHASE/CHASE_Test_8/masks/*')
        y_test.sort()

        test_dataset = Dataset(X_test, y_test)
        test_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=1,
            shuffle=False,
            pin_memory=True,
            drop_last=False)

        print('Dataset_name:', "stage1 test")
        tp_all = 0
        fp_all = 0
        fn_all = 0
        mean_iou = 0
        gts = []
        prs = []
        losses = []
        for i, pack in enumerate(test_loader, start=1):
            image, gt_ = pack
            # name = name[0]
            gt = gt_[0][0]
            gt = np.asarray(gt, np.float32)
            image = image.cuda()

            res, res2, res3, res4 = model(image)
            res = F.upsample(res, size=gt.shape, mode='bilinear', align_corners=False)
            loss = structure_loss_v2(res.cpu(), gt_)
            losses.append(loss.cpu().detach().numpy().squeeze())
            res = res.sigmoid().data.cpu().numpy().squeeze()
            res = (res - res.min()) / (res.max() - res.min() + 1e-8)
            pr = res.round()
           # pr = cv2.dilate(pr, kernel, iterations=1)
            #pr = cv2.erode(pr, kernel, iterations=1)
            # cv2.imwrite(os.path.join(save_path, dataset_name, name), res)
            gts.append(gt)
            prs.append(pr)
        mean_iou, mean_dice, mean_precision, mean_recall = get_scores(gts, prs)
#         writer.add_scalar(dataset_name + ' val loss',
#                             np.mean(losses),
#                             epoch )
#         writer.add_scalar(dataset_name + ' val dice',
#                             mean_dice,
#                             epoch )
#         writer.add_scalar(dataset_name + ' val iou',
#                             mean_iou,
#                             epoch )
    print("#"*20)

In [23]:
#from torchvision.transforms.functional import *
#import torchvision.transforms as transforms
from albumentations.augmentations.geometric import  resize,rotate
import albumentations.augmentations.crops.transforms as crop
import albumentations.augmentations.transforms as transforms
train_transform = Compose([
            rotate.RandomRotate90(),
            transforms.Flip(),
            #transforms.HueSaturationValue(),
            transforms.RandomBrightnessContrast(),
            transforms.GaussianBlur(),
            transforms.Transpose(),
            OneOf([
                crop.RandomCrop(224, 224, p=1),
                crop.CenterCrop(224, 224, p=1)
            ], p=0.2),
            resize.Resize(HEIGHT, HEIGHT)
        ], p=0.5)

In [12]:
train_img_paths = train_set#glob('../dataset/datasciencebowl/stage1_train/*')
train_dataset = Dataset(train_img_paths, [],transform =train_transform)
image,mask = train_dataset.__getitem__(random.randint(0,200))#read_image(train_img_paths[0]),read_mask(train_mask_paths[0])
#image = np.transpose(image,[1,2,0])
mask = np.transpose(mask,[1,2,0])
print(np.unique(mask),mask.shape)
import matplotlib.pyplot as plt
#plt.imshow(image[:,:,::-1])
#mask = cv2.imread(train_mask_paths[0])
plt.imshow((mask[:,:]*255).astype(np.uint8))
plt.show()
plt.imshow((np.transpose(image,[1,2,0])*255).astype(np.uint8))
plt.show()

NameError: name 'train_set' is not defined

In [25]:
from torch.utils.tensorboard import SummaryWriter

    # default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logs-chase/lan1')
if True:
        init_lr = 1e-4
        batchsize = 4
        trainsize_init = HEIGHT
        clip = 0.5
        num_epochs= 100
        train_save = 'chase-lan1'

        save_path = './snapshots/{}/'.format(train_save)
        if not os.path.exists(save_path):
            os.makedirs(save_path, exist_ok=True)
        else:
            print("Save path existed")
        #     sys.exit(1)


        log = pd.DataFrame(index=[], columns=[
            'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
        ])

        train_img_paths = glob('../dataset/CHASE/CHASE_Training_20/images/*')
        train_mask_paths = glob('../dataset/CHASE/CHASE_Training_20/masks/*')
        train_img_paths.sort()
        train_mask_paths.sort()

        train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)#train_transform
        train_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=batchsize,
            shuffle=True,
            pin_memory=True,
            drop_last=True
        )

        total_step = len(train_loader)
        model = IGH( backbone=dict(
                          type='mit_b2',
                          style='pytorch'), 
                      decode_head=None,
                      neck=None,
                      auxiliary_head=None,
                      train_cfg=dict(),
                        num_classes=1,
                    compound_coef=3,
                    ra = False,
                      test_cfg=dict(mode='whole'),
                      pretrained='pretrained/mit_b2.pth').cuda()
        #model.load_state_dict(torch.load(save_path+"last.pth")["state_dict"])
       # inference(model,None,0,test_set)
        # ---- flops and params ----
        params = model.parameters()
        optimizer = torch.optim.Adam(params, init_lr)#,weight_decay =1e-5
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                        T_max=len(train_loader)*num_epochs,
                                            eta_min=init_lr/1000)



        start_epoch = 1



        print("#"*20, f"Start Training", "#"*20)
        for epoch in range(start_epoch, num_epochs+1):

           # inference(model,writer,epoch)
            train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

            log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
                    train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
            ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
            log = log.append(log_tmp, ignore_index=True)
            log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
            writer.add_scalar('training loss',
                                    train_log['loss'].item(),
                                    epoch )
            writer.add_scalar('training dice',
                                    train_log['dice'].item(),
                                    epoch )
            writer.add_scalar('training iou',
                                    train_log['iou'].item(),
                                    epoch )

            if epoch %5==0:
                inference(model,writer,epoch,[])

Save path existed


2022-06-15 11:23:25,940 - mmseg - INFO - Use load_from_local loader
2022-06-15 11:23:25,993 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



#################### Start Training ####################
losstensor(3.7265, device='cuda:0'),   dice: tensor(0.1063, device='cuda:0')  ,iou:tensor(0.0562, device='cuda:0')2022-06-15 11:23:29.478094 Training Epoch [001/100], [loss: 3.7265   dice:0.1063  iou: 0.0562]
[Saving Checkpoint:] ./snapshots/chase-lan1/last.pth
losstensor(3.4864, device='cuda:0'),   dice: tensor(0.1731, device='cuda:0')  ,iou:tensor(0.0964, device='cuda:0')2022-06-15 11:23:33.268539 Training Epoch [002/100], [loss: 3.4864   dice:0.1731  iou: 0.0964]
[Saving Checkpoint:] ./snapshots/chase-lan1/last.pth
losstensor(3.2849, device='cuda:0'),   dice: tensor(0.4469, device='cuda:0')  ,iou:tensor(0.2897, device='cuda:0')2022-06-15 11:23:37.076600 Training Epoch [003/100], [loss: 3.2849   dice:0.4469  iou: 0.2897]
[Saving Checkpoint:] ./snapshots/chase-lan1/last.pth
losstensor(3.0324, device='cuda:0'),   dice: tensor(0.5766, device='cuda:0')  ,iou:tensor(0.4063, device='cuda:0')2022-06-15 11:23:40.939377 Training Epoch [

losstensor(1.8724, device='cuda:0'),   dice: tensor(0.7642, device='cuda:0')  ,iou:tensor(0.6190, device='cuda:0')2022-06-15 11:25:15.431205 Training Epoch [028/100], [loss: 1.8724   dice:0.7642  iou: 0.6190]
[Saving Checkpoint:] ./snapshots/chase-lan1/last.pth
losstensor(1.8110, device='cuda:0'),   dice: tensor(0.7749, device='cuda:0')  ,iou:tensor(0.6327, device='cuda:0')2022-06-15 11:25:19.278120 Training Epoch [029/100], [loss: 1.8110   dice:0.7749  iou: 0.6327]
[Saving Checkpoint:] ./snapshots/chase-lan1/last.pth
losstensor(1.7464, device='cuda:0'),   dice: tensor(0.7758, device='cuda:0')  ,iou:tensor(0.6342, device='cuda:0')2022-06-15 11:25:23.128631 Training Epoch [030/100], [loss: 1.7464   dice:0.7758  iou: 0.6342]
[Saving Checkpoint:] ./snapshots/chase-lan1/last.pth
####################
../dataset/IGH-small/train/train/*
Dataset_name: stage1 test
scores: dice=0.7051697097863444, miou=0.54515093667314, precision=0.6232517676210241, recall=0.8133713671961608
####################

losstensor(1.5324, device='cuda:0'),   dice: tensor(0.8059, device='cuda:0')  ,iou:tensor(0.6752, device='cuda:0')2022-06-15 11:27:02.240814 Training Epoch [055/100], [loss: 1.5324   dice:0.8059  iou: 0.6752]
[Saving Checkpoint:] ./snapshots/chase-lan1/last.pth
####################
../dataset/IGH-small/train/train/*
Dataset_name: stage1 test
scores: dice=0.742731325845961, miou=0.5913428669773362, precision=0.6717988211425341, recall=0.8320353214330406
####################
losstensor(1.4932, device='cuda:0'),   dice: tensor(0.8206, device='cuda:0')  ,iou:tensor(0.6960, device='cuda:0')2022-06-15 11:27:06.718043 Training Epoch [056/100], [loss: 1.4932   dice:0.8206  iou: 0.6960]
[Saving Checkpoint:] ./snapshots/chase-lan1/last.pth
losstensor(1.5708, device='cuda:0'),   dice: tensor(0.8067, device='cuda:0')  ,iou:tensor(0.6767, device='cuda:0')2022-06-15 11:27:10.595526 Training Epoch [057/100], [loss: 1.5708   dice:0.8067  iou: 0.6767]
[Saving Checkpoint:] ./snapshots/chase-lan1/last.pt

losstensor(1.4628, device='cuda:0'),   dice: tensor(0.8108, device='cuda:0')  ,iou:tensor(0.6823, device='cuda:0')2022-06-15 11:28:46.646869 Training Epoch [081/100], [loss: 1.4628   dice:0.8108  iou: 0.6823]
[Saving Checkpoint:] ./snapshots/chase-lan1/last.pth
losstensor(1.4453, device='cuda:0'),   dice: tensor(0.8133, device='cuda:0')  ,iou:tensor(0.6862, device='cuda:0')2022-06-15 11:28:50.617171 Training Epoch [082/100], [loss: 1.4453   dice:0.8133  iou: 0.6862]
[Saving Checkpoint:] ./snapshots/chase-lan1/last.pth
losstensor(1.4582, device='cuda:0'),   dice: tensor(0.8135, device='cuda:0')  ,iou:tensor(0.6860, device='cuda:0')2022-06-15 11:28:54.492815 Training Epoch [083/100], [loss: 1.4582   dice:0.8135  iou: 0.6860]
[Saving Checkpoint:] ./snapshots/chase-lan1/last.pth
losstensor(1.4214, device='cuda:0'),   dice: tensor(0.8275, device='cuda:0')  ,iou:tensor(0.7066, device='cuda:0')2022-06-15 11:28:58.350005 Training Epoch [084/100], [loss: 1.4214   dice:0.8275  iou: 0.7066]
[Savi

In [27]:
inference(model,writer,epoch,[])

####################
../dataset/IGH-small/train/train/*
Dataset_name: stage1 test
scores: dice=0.6600664353782312, miou=0.4931307319132958, precision=0.7565986197729992, recall=0.5867346129608122
####################


In [21]:
def rle_encoding(x):
    dots = np.where(x.T.flatten()==1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths
from skimage.morphology import label
def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

In [28]:
from glob import glob
import cv2
from keras.utils.np_utils import to_categorical 
kernel = np.ones((7,7), np.uint8)
files = glob('../dataset/datasciencebowl/stage2_test_final/*')
#model.load_state_dict(torch.load(save_path+"last.pth")['state_dict'])
#mobilenetv2_unet.load_weights("b4-wbifpn-81.11.h5")
#model.eval().cuda()
preds_upsampled=[]
test_files=[]

for file in files:
      name = file.split("/")[-1]
      path = file+"/images/"+name+".png"
      img = cv2.imread(path)
      
      h,w,_ = img.shape
      test_img = cv2.imread(path, cv2.IMREAD_COLOR)
      test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
      
      m = np.zeros((h,w))
      test_img,mask =  pad_image(test_img,m)
      test_img = cv2.resize(test_img,(HEIGHT,HEIGHT))
      p,p1,p2,p3 = model(torch.from_numpy(np.transpose(np.expand_dims((test_img/255.).astype(np.float32), axis=0),[0,3,1,2])).cuda())
      p = F.upsample(p, size=(HEIGHT,HEIGHT), mode='bilinear', align_corners=False)
      res = p.data.cpu().sigmoid().numpy().squeeze()#.sigmoid()
      res = (res - res.min()) / (res.max() - res.min() + 1e-8)
      pr = res.round()
      #print(pr.shape)
     # pr = np.transpose(pr,[1,2,0])
     # pr[pr>0.5] = 1
     # pr[pr<=0.5] = 0
      #p = pr  
      #print(pr.shape)
#       p = np.argmax(pr, axis=-1)
#      # print(p)
#       p = to_categorical(p, num_classes=3).astype(np.uint8)
      #print(print(p.shape))
      res = pr.copy().astype(np.uint8)*255
      res = cv2.resize(res,(w,h),interpolation=cv2.INTER_NEAREST)
      #res[:,:,0]=0
      res = cv2.erode(res, kernel, iterations=1) 
      res = cv2.dilate(res, kernel, iterations=1)
      res = res/255
      preds_upsampled.append(res)
      test_files.append(name)
      
#       break
# plt.imshow((test_img*255).astype(np.uint8))
# plt.show() 
# plt.imshow((res*255).astype(np.uint8))
# plt.show()      
#       res[:,:,2] = cv2.dilate(res[:,:,2], kernel, iterations=1)
#       res[:,:,2] = cv2.erode(res[:,:,2], kernel, iterations=1)    
      #print(np.max(res),np.min(res))
      #res = res[:,:,[1,0,2]]
      #res[p==1]=2
      #res[p==2]=1
      #cv2.imwrite("result/"+file.split("/")[-1].split(".")[0]+".png",(res).astype(np.uint8))
# rles = []
# new_test_ids = []
# for n, id_ in enumerate(test_files):
#     rle = list(prob_to_rles(preds_upsampled[n]))
#     rles.extend(rle)
#     new_test_ids.extend([id_] * len(rle))
# sub = pd.DataFrame()
# sub['ImageId'] = new_test_ids
# sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
# sub.to_csv('sub-dsbowl2018.csv', index=False)

In [29]:
rles = []
new_test_ids = []
for n, id_ in enumerate(test_files):
    rle = list(prob_to_rles(preds_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))


In [30]:
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('sub-dsbowl2018.csv', index=False)

In [ ]:
!scp output.csv thuan@172.23.0.6:~/ 


thuan@172.23.0.6's password: 

In [6]:
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                   compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
             compound_coef=4,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth')
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
from pthflops import count_ops
print(count_parameters(model))
x = torch.rand(1,3,384,384)
#count_ops(model, x,mode='jit')
from fvcore.nn import FlopCountAnalysis
flops = FlopCountAnalysis(model, x)
flops.total()

2022-06-05 18:32:26,970 - mmseg - INFO - Use load_from_local loader
2022-06-05 18:32:27,064 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



93489990


/home/s/thuannh/SegFormer_clean/mmseg/models/backbones/mix_transformer.py:98: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  q = self.q(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
/home/s/thuannh/SegFormer_clean/mmseg/models/backbones/mix_transformer.py:104: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  kv

50090527836

In [1]:
!pip install torch-summary

In [7]:
from torchsummary import summary


summary(model, (3, 512, 512))

Layer (type:depth-idx)                                  Output Shape              Param #
├─mit_b3: 1-1                                           [-1, 64, 128, 128]        --
|    └─OverlapPatchEmbed: 2-1                           [-1, 16384, 64]           --
|    |    └─Conv2d: 3-1                                 [-1, 64, 128, 128]        9,472
|    |    └─LayerNorm: 3-2                              [-1, 16384, 64]           128
|    └─ModuleList: 2                                    []                        --
|    |    └─Block: 3-3                                  [-1, 16384, 64]           314,880
|    |    └─Block: 3-4                                  [-1, 16384, 64]           314,880
|    |    └─Block: 3-5                                  [-1, 16384, 64]           314,880
|    └─LayerNorm: 2-2                                   [-1, 16384, 64]           128
|    └─OverlapPatchEmbed: 2-3                           [-1, 4096, 128]           --
|    |    └─Conv2d: 3-6                 

/home/s/anaconda3/envs/tf2.6/lib/python3.6/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Layer (type:depth-idx)                                  Output Shape              Param #
├─mit_b3: 1-1                                           [-1, 64, 128, 128]        --
|    └─OverlapPatchEmbed: 2-1                           [-1, 16384, 64]           --
|    |    └─Conv2d: 3-1                                 [-1, 64, 128, 128]        9,472
|    |    └─LayerNorm: 3-2                              [-1, 16384, 64]           128
|    └─ModuleList: 2                                    []                        --
|    |    └─Block: 3-3                                  [-1, 16384, 64]           314,880
|    |    └─Block: 3-4                                  [-1, 16384, 64]           314,880
|    |    └─Block: 3-5                                  [-1, 16384, 64]           314,880
|    └─LayerNorm: 2-2                                   [-1, 16384, 64]           128
|    └─OverlapPatchEmbed: 2-3                           [-1, 4096, 128]           --
|    |    └─Conv2d: 3-6                 